In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import ast
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler


df_ted_main = pd.read_csv("data/ted_main.csv")
df_ted_transcripts = df = pd.read_csv("data/transcripts.csv")

In [44]:
############### PREPROCESSING OF THE DATA ###############
# - removing columns that are not needed
# - one hot encoding columns
# - adding sentiment analysis of transcripts 
# - combining columns together to create a new structure of the data frame

In [45]:
# ----------------------------------------------------------
# 1. remove unwanted columns
# ----------------------------------------------------------

df_ted_main.drop(['description', 'film_date', 'main_speaker', 'name', 'num_speaker', 'related_talks', 'speaker_occupation', 'title', 'url'], axis=1, inplace=True)
df_ted_main.head()

,comments,duration,event,languages,published_date,ratings,tags,views
0,4553,1164,TED2006,60,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","['children', 'creativity', 'culture', 'dance',...",47227110
1,265,977,TED2006,43,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","['alternative energy', 'cars', 'climate change...",3200520
2,124,1286,TED2006,26,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","['computers', 'entertainment', 'interface desi...",1636292
3,200,1116,TED2006,35,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","['MacArthur grant', 'activism', 'business', 'c...",1697550
4,593,1190,TED2006,48,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","['Africa', 'Asia', 'Google', 'demo', 'economic...",12005869


In [46]:
# ----------------------------------------------------------
# 2. one hot encoding
# ----------------------------------------------------------

# one hot encoding tags

# iterate trough the data set
for i, row in df_ted_main.iterrows():
    # change string formed list into normal list
    element_list_form = ast.literal_eval(df_ted_main.at[i,'tags'])
    element_list_form = [i.strip() for i in element_list_form]
    # finish converting by replacing string version of a list for the list type
    df_ted_main.at[i,'tags'] = element_list_form

# one hot encoding with column named "tags"
# since "tags" is a Series on each row, we need to split them first and then we can do one hot encoding
df_ted_main = df_ted_main.drop('tags', 1).join(
    pd.get_dummies(
        pd.DataFrame(df_ted_main.tags.tolist()).stack()
    ).astype(int).sum(level=0)
)

df_ted_main.head()

,comments,duration,event,languages,published_date,ratings,views,3d printing,AI,AIDS,...,wikipedia,wind energy,women,women in business,work,work-life balance,world cultures,writing,wunderkind,youth
0,4553,1164,TED2006,60,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",47227110,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,265,977,TED2006,43,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",3200520,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,124,1286,TED2006,26,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",1636292,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,200,1116,TED2006,35,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...",1697550,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,593,1190,TED2006,48,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",12005869,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# one hot encoding "event" column

def one_hot_encode(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

df_ted_main = one_hot_encode(df_ted_main, 'event')
df_ted_main.head()

,comments,duration,languages,published_date,ratings,views,3d printing,AI,AIDS,Addiction,...,event_TEDxZurich,event_TEDxZurich 2011,event_TEDxZurich 2012,event_TEDxZurich 2013,event_Taste3 2008,event_The Do Lectures,event_Toronto Youth Corps,event_University of California,event_Web 2.0 Expo 2008,event_World Science Festival
0,4553,1164,60,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",47227110,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,265,977,43,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",3200520,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,124,1286,26,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",1636292,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,200,1116,35,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...",1697550,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,593,1190,48,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",12005869,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
#rating analysis

df_ted_main['ratings']=df_ted_main['ratings'].str.replace("'",'"')

df_ted_main=df_ted_main.merge(df_ted_main.ratings.apply(lambda x: pd.Series(pd.read_json(x)['count'].values,index=pd.read_json(x)['name'])), 
            left_index=True, right_index=True)

df_ted_main.drop('ratings', axis=1, inplace=True)

df_ted_main.head()

,comments,duration,languages,published_date,views,3d printing,AI,AIDS,Addiction,Africa,...,Longwinded,Confusing,Informative,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring
0,4553,1164,60,1151367060,47227110,0,0,0,0,0,...,387,242,7346,10581,300,10704,4439,1174,209,24924
1,265,977,43,1151367060,3200520,0,0,0,0,0,...,113,62,443,132,258,268,116,203,131,413
2,124,1286,26,1151367060,1636292,0,0,0,0,0,...,78,27,395,166,104,230,54,146,142,230
3,200,1116,35,1151367060,1697550,0,0,0,0,0,...,53,32,380,132,36,460,230,85,35,1070
4,593,1190,48,1151440680,12005869,0,0,0,0,1,...,110,72,5433,4606,67,2542,3736,248,61,2893


In [49]:
# sentiment analysis of transcript
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

df_ted_transcripts['polarity'] = df_ted_transcripts['transcript'].apply(pol)
df_ted_transcripts['subjectivity'] = df_ted_transcripts['transcript'].apply(sub)
df_ted_transcripts

,transcript,url,polarity,subjectivity
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,0.146452,0.462051
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,0.157775,0.424101
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,0.136579,0.475229
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,0.082928,0.439165
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,0.096483,0.450631
...,...,...,...,...
2462,"So, Ma was trying to explain something to me a...",https://www.ted.com/talks/duarte_geraldino_wha...,0.038878,0.352668
2463,This is a picture of a sunset on Mars taken by...,https://www.ted.com/talks/armando_azua_bustos_...,0.127087,0.538377
2464,"In my early days as a graduate student, I went...",https://www.ted.com/talks/radhika_nagpal_what_...,0.193538,0.526466
2465,I took a cell phone and accidentally made myse...,https://www.ted.com/talks/theo_e_j_wilson_a_bl...,0.100929,0.498050


In [50]:
# merging both datasets and delete the rows that don't have a transcipt
result = pd.merge(df_ted_main, df_ted_transcripts, left_index=True, right_index=True)

In [51]:
result

,comments,duration,languages,published_date,views,3d printing,AI,AIDS,Addiction,Africa,...,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring,transcript,url,polarity,subjectivity
0,4553,1164,60,1151367060,47227110,0,0,0,0,0,...,300,10704,4439,1174,209,24924,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,0.146452,0.462051
1,265,977,43,1151367060,3200520,0,0,0,0,0,...,258,268,116,203,131,413,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,0.157775,0.424101
2,124,1286,26,1151367060,1636292,0,0,0,0,0,...,104,230,54,146,142,230,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,0.136579,0.475229
3,200,1116,35,1151367060,1697550,0,0,0,0,0,...,36,460,230,85,35,1070,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,0.082928,0.439165
4,593,1190,48,1151440680,12005869,0,0,0,0,1,...,67,2542,3736,248,61,2893,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,0.096483,0.450631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,35,920,20,1496156399,1079698,0,1,0,0,0,...,9,103,11,26,4,258,"So, Ma was trying to explain something to me a...",https://www.ted.com/talks/duarte_geraldino_wha...,0.038878,0.352668
2463,34,452,17,1496242092,957803,0,0,0,0,0,...,11,74,21,30,4,185,This is a picture of a sunset on Mars taken by...,https://www.ted.com/talks/armando_azua_bustos_...,0.127087,0.538377
2464,18,1230,14,1496327443,874817,0,0,0,0,1,...,0,55,10,22,3,196,"In my early days as a graduate student, I went...",https://www.ted.com/talks/radhika_nagpal_what_...,0.193538,0.526466
2465,10,885,8,1496400857,946815,0,0,0,0,0,...,0,9,24,0,9,63,I took a cell phone and accidentally made myse...,https://www.ted.com/talks/theo_e_j_wilson_a_bl...,0.100929,0.498050


In [52]:
# deleting the transcript and url columns because we don't need them.
del result['transcript']
del result['url']

In [53]:
result

,comments,duration,languages,published_date,views,3d printing,AI,AIDS,Addiction,Africa,...,Informative,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring,polarity,subjectivity
0,4553,1164,60,1151367060,47227110,0,0,0,0,0,...,7346,10581,300,10704,4439,1174,209,24924,0.146452,0.462051
1,265,977,43,1151367060,3200520,0,0,0,0,0,...,443,132,258,268,116,203,131,413,0.157775,0.424101
2,124,1286,26,1151367060,1636292,0,0,0,0,0,...,395,166,104,230,54,146,142,230,0.136579,0.475229
3,200,1116,35,1151367060,1697550,0,0,0,0,0,...,380,132,36,460,230,85,35,1070,0.082928,0.439165
4,593,1190,48,1151440680,12005869,0,0,0,0,1,...,5433,4606,67,2542,3736,248,61,2893,0.096483,0.450631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,35,920,20,1496156399,1079698,0,1,0,0,0,...,88,120,9,103,11,26,4,258,0.038878,0.352668
2463,34,452,17,1496242092,957803,0,0,0,0,0,...,50,22,11,74,21,30,4,185,0.127087,0.538377
2464,18,1230,14,1496327443,874817,0,0,0,0,1,...,49,38,0,55,10,22,3,196,0.193538,0.526466
2465,10,885,8,1496400857,946815,0,0,0,0,0,...,13,43,0,9,24,0,9,63,0.100929,0.498050


In [54]:
scaler = MinMaxScaler() 
views = result['views']
result = result.loc[:, result.columns != 'views']
scaled_values = scaler.fit_transform(result) 
result.loc[:,:] = scaled_values

c:\python\python38\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [55]:
result

,comments,duration,languages,published_date,3d printing,AI,AIDS,Addiction,Africa,Alzheimer's,...,Informative,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring,polarity,subjectivity
0,0.710736,0.200937,0.833333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.750588,0.732401,0.136737,1.000000,0.301399,0.875466,0.153564,1.000000,0.424460,0.594066
1,0.040631,0.164421,0.597222,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.045264,0.009137,0.117593,0.025037,0.007876,0.151380,0.096253,0.016452,0.439007,0.545273
2,0.018597,0.224761,0.361111,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.040360,0.011490,0.047402,0.021487,0.003666,0.108874,0.104335,0.009109,0.411776,0.611009
3,0.030474,0.191564,0.486111,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.038827,0.009137,0.016408,0.042975,0.015617,0.063386,0.025716,0.042815,0.342850,0.564641
4,0.091889,0.206014,0.666667,0.000213,0.0,0.0,0.0,0.0,1.0,0.0,...,0.555124,0.318821,0.030538,0.237481,0.253666,0.184937,0.044820,0.115966,0.360264,0.579382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,0.004688,0.153290,0.277778,0.999242,0.0,1.0,0.0,0.0,0.0,0.0,...,0.008992,0.008306,0.004102,0.009623,0.000747,0.019389,0.002939,0.010232,0.286259,0.453431
2463,0.004532,0.061902,0.236111,0.999490,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005109,0.001523,0.005014,0.006913,0.001426,0.022371,0.002939,0.007303,0.399582,0.692199
2464,0.002032,0.213825,0.194444,0.999737,0.0,0.0,0.0,0.0,1.0,0.0,...,0.005007,0.002630,0.000000,0.005138,0.000679,0.016406,0.002204,0.007744,0.484953,0.676884
2465,0.000781,0.146456,0.111111,0.999950,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001328,0.002976,0.000000,0.000841,0.001630,0.000000,0.006613,0.002408,0.365977,0.640350


In [56]:
views

0       47227110
1        3200520
2        1636292
3        1697550
4       12005869
          ...   
2462     1079698
2463      957803
2464      874817
2465      946815
2466     1323235
Name: views, Length: 2467, dtype: int64

In [57]:
result['views'] = views

In [58]:
result

,comments,duration,languages,published_date,3d printing,AI,AIDS,Addiction,Africa,Alzheimer's,...,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring,polarity,subjectivity,views
0,0.710736,0.200937,0.833333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.732401,0.136737,1.000000,0.301399,0.875466,0.153564,1.000000,0.424460,0.594066,47227110
1,0.040631,0.164421,0.597222,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009137,0.117593,0.025037,0.007876,0.151380,0.096253,0.016452,0.439007,0.545273,3200520
2,0.018597,0.224761,0.361111,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011490,0.047402,0.021487,0.003666,0.108874,0.104335,0.009109,0.411776,0.611009,1636292
3,0.030474,0.191564,0.486111,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009137,0.016408,0.042975,0.015617,0.063386,0.025716,0.042815,0.342850,0.564641,1697550
4,0.091889,0.206014,0.666667,0.000213,0.0,0.0,0.0,0.0,1.0,0.0,...,0.318821,0.030538,0.237481,0.253666,0.184937,0.044820,0.115966,0.360264,0.579382,12005869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,0.004688,0.153290,0.277778,0.999242,0.0,1.0,0.0,0.0,0.0,0.0,...,0.008306,0.004102,0.009623,0.000747,0.019389,0.002939,0.010232,0.286259,0.453431,1079698
2463,0.004532,0.061902,0.236111,0.999490,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001523,0.005014,0.006913,0.001426,0.022371,0.002939,0.007303,0.399582,0.692199,957803
2464,0.002032,0.213825,0.194444,0.999737,0.0,0.0,0.0,0.0,1.0,0.0,...,0.002630,0.000000,0.005138,0.000679,0.016406,0.002204,0.007744,0.484953,0.676884,874817
2465,0.000781,0.146456,0.111111,0.999950,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002976,0.000000,0.000841,0.001630,0.000000,0.006613,0.002408,0.365977,0.640350,946815


In [59]:
#60% - train set, 20% - validation set, 20% - test set
train, validate, test = np.split(result.sample(frac=1), [int(.6*len(result)), int(.8*len(result))])

In [60]:
train

,comments,duration,languages,published_date,3d printing,AI,AIDS,Addiction,Africa,Alzheimer's,...,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring,polarity,subjectivity,views
726,0.094077,0.091584,0.638889,0.368920,0.0,0.0,0.0,0.0,0.0,0.0,...,0.053021,0.040109,0.080904,0.010592,0.099925,0.016165,0.027126,0.375634,0.552971,2934262
959,0.043132,0.170475,0.416667,0.454120,0.0,0.0,0.0,0.0,0.0,0.0,...,0.015989,0.010483,0.000934,0.012290,0.020880,0.020573,0.013282,0.409985,0.661041,1486880
1711,0.025785,0.090607,0.444444,0.709780,0.0,0.0,0.0,0.0,0.0,0.0,...,0.008652,0.020966,0.007567,0.003055,0.079791,0.022777,0.027286,0.384361,0.572598,3133189
684,0.070167,0.110135,0.444444,0.354649,0.0,0.0,0.0,0.0,0.0,0.0,...,0.029902,0.022789,0.114350,0.007061,0.056674,0.011756,0.053128,0.495412,0.671714,2465179
866,0.011564,0.075766,0.486111,0.422862,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004153,0.000912,0.001588,0.003395,0.010440,0.001470,0.011958,0.455474,0.608168,348647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,0.022504,0.136302,0.444444,0.843750,0.0,0.0,0.0,0.0,0.0,0.0,...,0.029695,0.032361,0.008128,0.001630,0.283371,0.030125,0.017134,0.481761,0.616903,3098476
771,0.055321,0.181605,0.458333,0.387456,0.0,0.0,0.0,0.0,0.0,0.0,...,0.070188,0.023701,0.044189,0.006586,0.222222,0.011756,0.056218,0.391915,0.545971,3951683
276,0.016253,0.338410,0.291667,0.196571,0.0,0.0,0.0,0.0,0.0,0.0,...,0.055306,0.020966,0.003176,0.107482,0.067114,0.013960,0.002648,0.438377,0.833939,4204848
972,0.013596,0.040227,0.500000,0.458131,0.0,0.0,0.0,0.0,0.0,0.0,...,0.013774,0.013218,0.004671,0.001901,0.035794,0.001470,0.019582,0.381615,0.489737,704205


In [61]:
validate

,comments,duration,languages,published_date,3d printing,AI,AIDS,Addiction,Africa,Alzheimer's,...,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring,polarity,subjectivity,views
1438,0.031411,0.148409,0.444444,0.608133,0.0,0.0,0.0,0.0,0.0,0.0,...,0.028518,0.005014,0.002055,0.017653,0.030574,0.016165,0.009309,0.360610,0.696095,1229865
821,0.119081,0.034954,0.541667,0.404803,0.0,0.0,0.0,0.0,0.0,0.0,...,0.020281,0.369644,0.062593,0.024375,0.133482,0.193240,0.047510,0.359190,0.490426,2152734
2316,0.019222,0.135716,0.263889,0.946660,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006368,0.025068,0.028401,0.003259,0.025354,0.025716,0.016813,0.419408,0.694254,1181475
2341,0.005782,0.157977,0.319444,0.955937,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009483,0.003646,0.013920,0.002920,0.017897,0.003674,0.035071,0.303550,0.596249,1290086
1713,0.063135,0.028705,0.430556,0.710034,0.0,0.0,0.0,0.0,0.0,0.0,...,0.055444,0.064266,0.003924,0.010864,0.140194,0.012491,0.008387,0.387642,0.613241,1736329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650,0.014690,0.152314,0.388889,0.687761,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004845,0.002279,0.009903,0.001086,0.020134,0.006613,0.012359,0.383838,0.659805,1514912
817,0.021097,0.083187,0.375000,0.403043,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001384,0.059253,0.009716,0.000883,0.090977,0.216752,0.000762,0.374311,0.513758,452578
937,0.021566,0.171256,0.430556,0.447116,0.0,0.0,0.0,0.0,0.0,0.0,...,0.030802,0.002735,0.004017,0.019690,0.024609,0.002204,0.023835,0.336903,0.531460,1586547
744,0.100641,0.062097,0.000000,0.378770,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003184,0.040109,0.072590,0.006450,0.074571,0.031594,0.027728,0.328204,0.575624,292395


In [62]:
test

,comments,duration,languages,published_date,3d printing,AI,AIDS,Addiction,Africa,Alzheimer's,...,Fascinating,Unconvincing,Persuasive,Jaw-dropping,OK,Obnoxious,Inspiring,polarity,subjectivity,views
1976,0.016253,0.188049,0.319444,0.812197,0.0,0.0,0.0,0.0,0.0,0.0,...,0.028172,0.008660,0.018311,0.001765,0.098434,0.018369,0.012520,0.436817,0.699974,4597270
2308,0.005157,0.100762,0.277778,0.943906,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002215,0.005014,0.002522,0.000475,0.023117,0.003674,0.002247,0.381570,0.635674,859817
622,0.059072,0.204648,0.513889,0.335121,0.0,0.0,0.0,0.0,0.0,0.0,...,0.093930,0.014129,0.050635,0.023153,0.073826,0.019838,0.091770,0.474889,0.567767,4375111
1186,0.055634,0.152314,0.472222,0.525743,0.0,0.0,0.0,0.0,0.0,0.0,...,0.035578,0.015497,0.029802,0.021727,0.024609,0.032329,0.030416,0.443804,0.684590,1803069
567,0.085482,0.122437,0.458333,0.315259,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005261,0.005014,0.055680,0.040060,0.022371,0.008082,0.053128,0.327809,0.534761,2543750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,0.220034,0.099590,0.472222,0.612138,0.0,0.0,0.0,0.0,0.0,0.0,...,0.021665,0.002279,0.066984,0.014598,0.021626,0.007348,0.094017,0.361377,0.534283,3005687
1483,0.059853,0.135325,0.430556,0.623895,0.0,0.0,0.0,0.0,0.0,0.0,...,0.038555,0.020055,0.029895,0.003666,0.038777,0.011021,0.004735,0.303788,0.700477,2162439
1187,0.075168,0.266745,0.305556,0.525991,0.0,0.0,0.0,0.0,0.0,0.0,...,0.028518,0.025068,0.013173,0.029400,0.039523,0.057311,0.029493,0.487983,0.625480,1618434
2076,0.019378,0.118141,0.527778,0.854518,0.0,0.0,0.0,0.0,0.0,0.0,...,0.020073,0.088423,0.033539,0.002716,0.357196,0.024982,0.014526,0.330252,0.617310,9163813
